In [1]:
import json
from enum import Enum
from collections import namedtuple
from ipynb.fs.full.Critical_Section import *
from ipynb.fs.full.CPv1 import *
import numpy as np

In [2]:
class lock_event:
    def __init__(self,n_thr):
        self.lock_clock = np.zeros(n_thr)
        self.objects = {}

class lock_map:
    def __init__(self,init_vc):
        self.lock_vc = init_vc
        self.read_objects = {}
        self.write_objects = {}
    
    def __str__(self):
        return "Main Clock : %s\nObject Read Clocks : %s\nWrite Clocks : %s\n" % (self.lock_vc,self.read_objects,self.write_objects)
    
class object_vc:
    def __init__(self,l_vc,tid):
        self.vc = l_vc
        self.tid = tid

In [18]:
def vc_cp_ordered(vc1,vc2): # Checks if vc1 CPs vc2. Returns true if vc2 does not CP vc1
    r = False
    ct = 0
    for i in range(len(vc1)):
        if vc2[i] > vc1[i]:
            return True
        else:
            ct+=1
    return ct==len(vc1)

def compare_vc_to_other_locks(evt,compare_map,v_vc,t_stk):
    ret = False
    for k in compare_map.keys():
        if k == evt.var_name: # Check if currently accessed under any lock and earlier accessed whithout any lock
#             print("Empty",v_vc,compare_map[evt.var_name])
            if evt.tid != compare_map[evt.var_name].tid:
                ret = ret or vc_cp_ordered(v_vc,compare_map[evt.var_name].vc)
        elif isinstance(compare_map[k],lock_map) and k not in t_stk:
            if (evt.e_type == events_list.Read.value and evt.var_name in compare_map[k].write_objects.keys()) or evt.var_name in compare_map[k].write_objects.keys():
#                 print("Lock",v_vc,compare_map[k].accessed_objects[v_name],t_stk,k)
                
                if evt.tid != compare_map[k].write_objects[evt.var_name].tid:
                    ret = ret or vc_cp_ordered(v_vc,compare_map[k].write_objects[evt.var_name].vc)
            elif evt.e_type == events_list.Write.value and evt.var_name in compare_map[k].read_objects.keys():
                if evt.tid != compare_map[k].read_objects[evt.var_name].tid:
                    ret = ret or vc_cp_ordered(v_vc,compare_map[k].read_objects[evt.var_name].vc)
#         elif k == evt.var_name and len(t_stk) == 0 :
# #             print("Empty",v_vc,compare_map[evt.var_name])
#             ret = ret or vc_cp_ordered(v_vc,compare_map[evt.var_name].vc)
    return ret

def add_current_vc(evt, t_map, t_vc, l_stk,s_l_map):
#     if evt.var_name in l_map.keys():
#         l_map[evt.var_name].append(t_vc)
#     else:
    if len(l_stk) == 0:
        s_l_map[evt.var_name] = object_vc(t_vc,evt.tid) #t_vc
#         print(evt.var_name," : ",l_map[evt.var_name])
    else:
        if evt.e_type == events_list.Read.value:
            s_l_map[l_stk[-1]].read_objects[evt.var_name] = object_vc(t_vc,evt.tid)
            t_map.read_objects[evt.var_name] = object_vc(t_vc,evt.tid)
        elif evt.e_type == events_list.Write.value:
            s_l_map[l_stk[-1]].write_objects[evt.var_name] = object_vc(t_vc,evt.tid)
            t_map.write_objects[evt.var_name] = object_vc(t_vc,evt.tid)
#         print(l_stk[-1]+" : ")
#         print(l_map)
        
def get_vc_var(v_name,t_vc,l_map,t_stk):
    if len(t_stk) == 0:
        return t_vc
    else:
        return l_map.lock_vc

### Read the events from log

In [19]:
# events, critical_section_list,num_threads,lock_access_list = gen_event_cs2("temp.log")
# cp_matrix_v2 = pass2(events,critical_section_list,lock_access_list)

# for x in critical_section_list:
#     print(x)

# print(num_threads,lock_access_list)

In [23]:
def update_clocks_temp(t_map,main_map,e_idx,cs_idx_list):
    for cs_idx in cs_idx_list:
        if e_idx in critical_section_list[cs_idx].event_idx:
            for x in critical_section_list[cs_idx].read_objects:
                if x in main_map.read_objects.keys():
                    t_map.read_objects[x] = main_map.read_objects[x]
                else:
                    t_map.read_objects[x] = object_vc(t_map.lock_vc,events[e_idx].tid)
            for x in critical_section_list[cs_idx].write_objects:
                if x in main_map.write_objects.keys():
                    t_map.write_objects[x] = main_map.write_objects[x]
                else:
                    t_map.write_objects[x] = object_vc(t_map.lock_vc,events[e_idx].tid)
                    
def check_rule1(evt,evt_idx):
    cs_idx_list = 0
    for cs_idx in lock_access_list[evt.var_name]:
        if evt_idx in critical_section_list[cs_idx].event_idx:
            current_cs_idx = cs_idx
            break
        cs_idx_list+=1
    
    if cs_idx_list>0:
#         print("Check Rule 1 : ",critical_section_list[cs_idx],critical_section_list[lock_access_list[evt.var_name][cs_idx_list-1]])
        if is_cp(cs_idx,lock_access_list[evt.var_name][cs_idx_list-1],critical_section_list) or critical_section_list[cs_idx].tid == critical_section_list[lock_access_list[evt.var_name][cs_idx_list-1]].tid:
            return True

def merge_maps(map_to,map_from):
    for k in map_from.read_objects.keys():
        map_to.read_objects[k] = map_from.read_objects[k]
    for k in map_from.write_objects.keys():
        map_to.write_objects[k] = map_from.write_objects[k]
    map_to.lock_vc = map_from.lock_vc

In [28]:
def CP_V2(num_threads,lock_access_list):
    thread_lock_stack = []
    thread_map_stack = []
    thread_vc = np.zeros((num_threads,num_threads))
    for i in range(num_threads):
        thread_lock_stack.append([])
        thread_map_stack.append([])
    evt_idx = 0
    super_locks_map = {}

    for e in events:
#         print("Index : ",evt_idx)
        if e.e_type == events_list.Read.value or e.e_type == events_list.Write.value:
            if len(thread_lock_stack[e.tid]) == 0:
                var_vc = get_vc_var(e.var_name,thread_vc[e.tid],super_locks_map,thread_lock_stack[e.tid])
                if compare_vc_to_other_locks(e,super_locks_map,var_vc,thread_lock_stack[e.tid]):
                    print("Index : ",evt_idx)
                    print("DR detected at event : ",e)
                add_current_vc(e,None,var_vc,thread_lock_stack[e.tid],super_locks_map)
            else:
                temp_map = thread_map_stack[e.tid][-1]
                var_vc = get_vc_var(e.var_name,thread_vc[e.tid],temp_map,thread_lock_stack[e.tid])
                if compare_vc_to_other_locks(e,super_locks_map,var_vc,thread_lock_stack[e.tid]):
                    print("Index : ",evt_idx)
                    print("DR detected at event : ",e)
                add_current_vc(e,temp_map,var_vc,thread_lock_stack[e.tid],super_locks_map)
                thread_map_stack[e.tid][-1] = temp_map
        elif e.e_type == events_list.Acquire.value:
            thread_lock_stack[e.tid].append(e.var_name)
            if e.var_name not in super_locks_map.keys():
                temp_map = lock_map(thread_vc[e.tid])#super_locks_map[e.var_name]
                super_locks_map[e.var_name] = temp_map
                update_clocks_temp(temp_map,super_locks_map[e.var_name],evt_idx,lock_access_list[e.var_name])
            else:
                temp_map = lock_map(super_locks_map[e.var_name].lock_vc)
                update_clocks_temp(temp_map,super_locks_map[e.var_name],evt_idx,lock_access_list[e.var_name])
            if len(thread_lock_stack[e.tid]) > 1: # Update the lock VC of the child clock to parents clock
                if vc_cp_ordered(temp_map.lock_vc,super_locks_map[thread_lock_stack[e.tid][-2]].lock_vc):
                    temp_map.lock_vc = super_locks_map[thread_lock_stack[e.tid][-2]].lock_vc
            if check_rule1(e,evt_idx):
                temp_map.lock_vc[e.tid]+=1 # cp_clock_increment
            thread_map_stack[e.tid].append(temp_map)
    #         print("Temp Map : ")
    #         print(temp_map)
        elif e.e_type == events_list.Release.value:
            popped_lock = thread_lock_stack[e.tid].pop()
            if popped_lock in super_locks_map:
                merge_maps(super_locks_map[popped_lock],thread_map_stack[e.tid].pop())
            else:
                super_locks_map[popped_lock] = thread_map_stack[e.tid].pop()

            if len(thread_lock_stack[e.tid]) > 0:
                merge_maps(thread_map_stack[e.tid][-1],super_locks_map[popped_lock])
            else:
                thread_vc[e.tid] = super_locks_map[popped_lock].lock_vc
        evt_idx+=1
    #     print("Locks Map : ")
    #     for k in super_locks_map.keys():
    #         print(k," : ",super_locks_map[k])
    #     print("\n")

In [29]:
# events, critical_section_list,num_threads,lock_access_list = gen_event_cs2("TestCases/Figure9a.log")
events, critical_section_list,num_threads,lock_access_list = gen_event_cs2("temp.log")
for x in critical_section_list:
    print(x)
# print(num_threads,lock_access_list)
CP_V2(num_threads,lock_access_list)

Thread : 1 
Locks : set() 
Read Objects : {'phi_x'} 
Write Objects : set() 
Events : {0} 
Child Sections : set()

Thread : 1 
Locks : set() 
Read Objects : {'phi_y'} 
Write Objects : set() 
Events : {1} 
Child Sections : set()

Thread : 1 
Locks : {'l', 'n', 'm'} 
Read Objects : {'x', 'xl', 'a'} 
Write Objects : {'y', 'yl', 'b'} 
Events : {2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17} 
Child Sections : {3, 4, 5}

Thread : 1 
Locks : {'l', 'n'} 
Read Objects : {'x', 'xl'} 
Write Objects : {'y', 'yl'} 
Events : {5, 6, 7, 8, 9, 10, 11, 12} 
Child Sections : {4}

Thread : 1 
Locks : {'n'} 
Read Objects : {'xl'} 
Write Objects : {'yl'} 
Events : {8, 9, 10, 11} 
Child Sections : set()

Thread : 1 
Locks : {'n'} 
Read Objects : {'xl'} 
Write Objects : {'yl'} 
Events : {16, 13, 14, 15} 
Child Sections : set()

Main Clock : [0. 1.]
Object Read Clocks : {'x': <__main__.object_vc object at 0x7f21d177ada0>, 'xl': <__main__.object_vc object at 0x7f21d16cb6a0>}
Write Clocks : {'y': <__main

In [26]:
import glob
for filename in glob.glob("TestCases/*.log"):
    print(filename,"--------------------------------------------------------------------------")
    events, critical_section_list,num_threads,lock_access_list = gen_event_cs2(filename)
    CP_V2(num_threads,lock_access_list)

TestCases/Figure1.log
Index :  0
Index :  1
Index :  2
Index :  3
Index :  4
Index :  5
Index :  6
Index :  7
Index :  7
DR detected at event :  event_raw(vc_w='[(0:0) (1:1)]', vc_r='[(0:0) (1:0) (2:3)]', vc='[(0:1) (1:2) (2:3)]', tid=2, e_type=0, var_name='test/Figure1_x')
TestCases/TestCase2.log
Index :  0
Index :  1
Index :  1
DR detected at event :  event_raw(vc_w='', vc_r='', vc='[(0:1) (1:2) (2:3)]', tid=2, e_type=1, var_name='phi_x')
TestCases/TestCase6.log
Index :  0
Index :  1
Index :  2
Index :  3
Index :  4
Index :  4
DR detected at event :  event_raw(vc_w='', vc_r='', vc='[(0:1) (1:2) (2:3)]', tid=2, e_type=1, var_name='phi_x')
Index :  5
TestCases/TestCase4.log
Index :  0
Index :  1
Index :  2
Index :  2
DR detected at event :  event_raw(vc_w='', vc_r='', vc='[(0:1) (1:2) (2:3)]', tid=2, e_type=1, var_name='phi_x')
Index :  3
TestCases/Figure3.log
Index :  0
Index :  1
Index :  2
Index :  3
Index :  4
Index :  5
Index :  6
Index :  7
Index :  8
Index :  9
Index :  10
Main 